# CS Department Comparison

## Data Collection

### Class Sizes Data

As a UMD student, we have [Testudo](https://app.testudo.umd.edu/soc/) which lists our schedule of classes with their associated class sizes. I can scrape this page to get the CS class sizes at UMD. I assumed other public universities would have similar systems but they would all be different and would take time for me to learn. To solve this problem, I made several Reddit posts asking for similar systems to Testudo at other universities on their associated subreddits. Students on Reddit were very helpful and gave me tips on how to get what I was looking for, and this saved me a lot of time.

I had time to scrape data for 5 universities, and I plan to expand this list eventually:

- University of Maryland
- Georgia Tech
- University of Washington
- University of Michigan
- University of California Berkeley

#### University of Maryland

UMD students created a website called PlanetTerp which has course reviews, instructor reviews, and grade distribution data. I used their public API to get CS course enrollment information for the fall 2021 semester. Their api documentation can be found here: https://api.planetterp.com/#planetterp-api

The first step was getting a list of CS courses offered by UMD, and I did this by calling their "courses" endpoint and searching for CS classes which are prefixed with "CMSC" at UMD. I changed the default class return limit to 300 because UMD has offered over 100 different CS courses over the years. I also extract the professors which have taught each of these courses over the years because this will help me determine section sizes for the fall 2021 semester.

I then use PlanetTerp's "grades" endpoint to get the course grades for the fall 2021 semester of a certain course with a certain professor. If that professor wasn't teaching that semester but has taught in the past, the api would return 0 for them. This endpoint returns all the sections of that professor, which for some courses is the sizes of a bunch of their discussions. Some cleaning of the data needs to be done to get their lecture size.

I will not be running and showing the output of the scraper below since it takes a while to complete. The "umd.csv" generated will be shown in the following code cell.

In [ ]:
import requests
from time import sleep
import pandas as pd


# get list of cs courses
params = {"department": "CMSC", "limit": 300}
cs_courses_data = requests.get("https://api.planetterp.com/v1/courses", params=params).json()

cs_course_nums = []

for course in cs_courses_data:
    cs_course_nums.append(course['department'] + str(course['course_number']))

# list of professors for each course so can get grades for each professor of the
# course during a specific semester to see how big lectures arec
course_professors = []

for course in cs_courses_data:
    course_professors.append(course["professors"])

class_names = []
class_sizes = []
class_profs = []

# get grades for each course in dictionary
semester = "202108" # fall 2021
for i, course in enumerate(cs_course_nums):
    for prof in course_professors[i]:
        params = {"course": course, "semester": semester, "professor": prof}

        # for each course returns a list of sections with corresponding grade data
        sections = requests.get("https://api.planetterp.com/v1/grades", params=params).json()

        print("{} {}".format(course, prof))
        for section in sections:
            # didn't get back a dictionary on the api call for some reason so skip
            # the reason seems to be inconsistent naming of professors so some
            # names of the same professor return nothing
            if not type(section) is dict:
                print("skipping")
                continue
            total_students_section = 0
            # convert section dictionary into list so only preserve items that have grade counts
            section_grades = list(section.items())[4:]
            for grade in section_grades:
                # add number of students who got this grade in this section to total students in section
                total_students_section += grade[1]
            
            # append section info to lists
            class_names.append(course)
            class_profs.append(prof)
            class_sizes.append(total_students_section)

        sleep(0.5)

# create dataframe from course counts
data_dict = {"course_name": class_names, "professor": class_profs, "count": class_sizes}

courses_df = pd.DataFrame.from_dict(data_dict)

courses_df.to_csv("umd.csv")


In [3]:
import pandas as pd
umd_data = pd.read_csv("class_sizes_data/umd.csv")
umd_data

,Unnamed: 0,course_name,professor,count
0,0,CMSC100,Jessica Long,33
1,1,CMSC100,Corie Brown,17
2,2,CMSC100,Veronica Sanchez,19
3,3,CMSC106,Jandelyn Plane,51
4,4,CMSC122,Pedram Sadeghian,83
...,...,...,...,...
183,183,CMSC829A,Erin Molloy,21
184,184,CMSC838G,Leonidas Lampropoulos,11
185,185,CMSC848B,Christopher Metzler,21
186,186,CMSC858G,Michael Gullans,28


##### Cleaning UMD Data

The list of discussion sections for each professor needs to be combined by professor and course to get the lecture size of that course for that professor.

In [9]:
classes_df = pd.read_csv("class_sizes_data/umd.csv")

# if row course_name and professor are the same, combine rows into one row

grouped = classes_df.groupby(["course_name", "professor"])

umd = grouped.agg({"count": "sum"})

umd

count
course_name professor                   
CMSC100     Corie Brown               17
            Jessica Long              33
            Veronica Sanchez          19
CMSC106     Jandelyn Plane            51
CMSC122     Jennifer Manly            61
...                                  ...
CMSC829A    Erin Molloy               21
CMSC838G    Leonidas Lampropoulos     11
CMSC848B    Christopher Metzler       21
CMSC858G    Michael Gullans           28
CMSC858O    Xiaodi Wu                 12

[82 rows x 1 columns]

#### Georgia Tech

Georgia Tech has a public class registry called Oscar located at:  https://oscar.gatech.edu/. The user can search for all CS courses in a specific term and Oscar will generate a static HTML page response that can be scraped for data. However, it takes a long time for this query to execute so I saved this HTML page locally to then be loaded by my program later. I used the Python package [Uncurl](https://github.com/spulec/uncurl) to generate the appropriate POST request. My code to get the HTML is below:

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
import json

res = requests.post("https://oscar.gatech.edu/bprod/bwckschd.p_get_crse_unsec",
    data='term_in=202108&sel_subj=dummy&sel_day=dummy&sel_schd=dummy&sel_insm=dummy&sel_camp=dummy&sel_levl=dummy&sel_sess=dummy&sel_instr=dummy&sel_ptrm=dummy&sel_attr=dummy&sel_subj=CS&sel_crse=&sel_title=&sel_schd=%25&sel_from_cred=&sel_to_cred=&sel_camp=A&sel_ptrm=%25&sel_instr=%25&sel_attr=%25&begin_hh=0&begin_mi=0&begin_ap=a&end_hh=0&end_mi=0&end_ap=a',
    headers={
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.5",
        "Connection": "keep-alive",
        "Content-Type": "application/x-www-form-urlencoded",
        "DNT": "1",
        "Origin": "https://oscar.gatech.edu",
        "Referer": "https://oscar.gatech.edu/bprod/bwckgens.p_proc_term_date",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0"
    },
    cookies={
        "BIGipServer~BANNER~oscar.coda_443": "572230154.64288.0000"
    },
    auth=(),
)

# save response in html so don't have to do 10 sec query every time
with open("gatech.html", "w") as f:
    f.write(res.text)

I have the HTML page which lists all of the CS classes for the fall 2021 term, great! But while the page lists the classes, you have to click on each class to get to how many seats are in that class. This means I'll have to make a program which gets all the links, goes to each link, and extracts the appropriate class data from each class page. The code to do that is below but won't be run because it takes a while.

In [ ]:
# link to class with seats is of form bprod/bwckschd.p_disp_detail_sched?term_in=202108&crn_in=93519
# get all links on page and filter for links like this

f = open("gatech.html", "r")

soup = BeautifulSoup(f, 'html.parser')

links = soup.find_all('a')

class_links = []
class_titles = []

for link in links:
    if link.get('href') != None and "bprod/bwckschd.p_disp_detail_sched?term_in=202108" in link.get('href'):
        class_links.append(link.get('href'))
        class_titles.append(link.get_text())

f.close()

class_data = {"title": class_titles, "link": class_links}

classes_df = pd.DataFrame.from_dict(class_data)

# print(classes_df)

# extracting class name from title which has other junk

classes_df["class_name"] = ""

for index, row in classes_df.iterrows():
    class_name = re.findall("CS \d{4}", row['title'])[0]
    classes_df.at[index, "class_name"] = class_name

# when adding up class counts, make sure to exclude those which are:
# - recitations
# - studios
# - supervised/unsupervised laboratory
# - directed study
# - dissertation
# - thesis
# once go to link, can find class type on page, credit number
# basically only look for lecture type with credits > 0, which can be found on class page


# add columns to classes_df
# - type
# - credits
# - seat_capacity
# - filled_seats

classes_df["type"] = ""
classes_df["credits"] = 0.0
classes_df["seat_capacity"] = 0
classes_df["filled_seats"] = 0

# list of links that had problems:
# will save them to a file after program over
problem_links = []

# go to all the links and fill in these two columns
for index, row in classes_df.iterrows():
    print(row["class_name"])

    link = "https://oscar.gatech.edu" + row["link"]
    res = requests.get(link)

    class_soup = BeautifulSoup(res.content, 'html.parser')

    # page is mostly made up of tables
    tables = class_soup.find_all('table')
    # assuming that the table of interest will always be at index 3, could be problems here
    # if not always true...
    data_table = tables[3]

    match_class_type = re.search("(\w+)\* Schedule Type", data_table.text)
    if match_class_type:
        classes_df.at[index, "type"] = match_class_type.group(1)
    else:
        print("for {} was unable to find class type".format(link))
        problem_links.append(link)

    match_class_credits = re.search("(\d\.\d{3}) Credits", data_table.text)
    if match_class_credits:
        classes_df.at[index, "credits"] = match_class_credits.group(1)
    else:
        print("for {} was unable to find class credits".format(link))
        problem_links.append(link)

    match_class_seats = re.search("Seats\n(\d+)\n(\d+)\n", data_table.text)
    if match_class_seats:
        classes_df.at[index, "seat_capacity"] = match_class_seats.group(1)
        classes_df.at[index, "filled_seats"] = match_class_seats.group(2)
    else:
        print("for {} was unable to find class seats".format(link))
        problem_links.append(link)

    # update at every iteration so no data lost
    classes_df.to_csv("gatech.csv")
    
    sleep(2)
   

classes_df.to_csv("gatech.csv")

# writing problem links list to json in case have to examine later
with open("problem_links.json", "w") as f:
    f.write(json.dumps(problem_links))


Each class's data will be written to a dataframe. Also, if a link doesn't work I add that to a file that I can inspect after the program ends to do some detective work on what went wrong. Below is what the above program generates.

In [8]:
gatech = pd.read_csv("class_sizes_data/gatech.csv")
# unnamed column generated for some reason so delete it
gatech = gatech.loc[:, ~gatech.columns.str.contains('^Unnamed')]
gatech

,title,link,class_name,type,credits,seat_capacity,filled_seats
0,Freshman Leap Seminar - 87695 - CS 1100 - A1,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 1100,Lecture,1.0,200,171
1,Freshman Leap Seminar - 87696 - CS 1100 - A2,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 1100,Lecture,1.0,240,240
2,Freshman Leap Seminar - 89255 - CS 1100 - A3,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 1100,Lecture,1.0,125,125
3,Freshman Leap Seminar - 94424 - CS 1100 - A4,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 1100,Lecture,1.0,280,260
4,Freshman Leap Seminar - 92598 - CS 1100 - B1,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 1100,Lecture,1.0,240,239
...,...,...,...,...,...,...,...
1441,Doctoral Thesis - 93112 - CS 9000 - W13,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 9000,Dissertation,1.0,10,1
1442,Doctoral Thesis - 81156 - CS 9000 - X01,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 9000,Dissertation,1.0,15,1
1443,Doctoral Thesis - 95063 - CS 9000 - X03,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 9000,Dissertation,1.0,10,0
1444,Doctoral Thesis - 94765 - CS 9000 - Y04,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 9000,Dissertation,1.0,10,1


##### Cleaning Georgia Tech Data

Some of the classes listed are clearly not lectures, so I want to filter to classes which are "Lecture" and have > 0.0 credits.

In [19]:
gatech = gatech[(gatech["type"] == "Lecture") & (gatech["credits"] > 0.0)]
gatech = gatech.reset_index(drop=True)
gatech

,title,link,class_name,type,credits,seat_capacity,filled_seats
0,Freshman Leap Seminar - 87695 - CS 1100 - A1,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 1100,Lecture,1.0,200,171
1,Freshman Leap Seminar - 87696 - CS 1100 - A2,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 1100,Lecture,1.0,240,240
2,Freshman Leap Seminar - 89255 - CS 1100 - A3,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 1100,Lecture,1.0,125,125
3,Freshman Leap Seminar - 94424 - CS 1100 - A4,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 1100,Lecture,1.0,280,260
4,Freshman Leap Seminar - 92598 - CS 1100 - B1,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 1100,Lecture,1.0,240,239
...,...,...,...,...,...,...,...
172,Special Topics - 93832 - CS 8803 - PHI,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 8803,Lecture,3.0,30,20
173,Special Topics - 87122 - CS 8803 - PP,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 8803,Lecture,3.0,15,12
174,Special Topics - 93244 - CS 8803 - SG,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 8803,Lecture,3.0,10,8
175,Special Topics - 92061 - CS 8803 - SMC,/bprod/bwckschd.p_disp_detail_sched?term_in=20...,CS 8803,Lecture,3.0,40,16


#### University of Washington

Similarly to Georgia Tech, the University of Washington's class registry generates an HTML file which can be scraped for CS class sizes. This time, the class sizes are listed directly under the course. However, lecture sizes and discussion sizes are listed in the same area, so some logic has to be applied to only extract the lecture sizes. Also, the HTML page is generated in a somewhat difficult way for parsing.

I had to get distinct classes by splitting on a class header element. Between class header elements were tables which listed lecture and discussion sizes. These are the associated tables for the class that I would parse to extract lecture sizes. To differentiate between lecture and discussion, I would check if the "credits" in the table for the discussion/lecture is numeric, and if it was numeric that would mean that that table was associated with a lecture. I would then use regex to get the class size.

Below is the code which generates the csv file for Washington.

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

res = requests.get("https://www.washington.edu/students/timeschd/AUT2021/cse.html")

soup = BeautifulSoup(res.content, 'html.parser')

# tables will be table elements with class headings
# class enrollment information in tables between headings tables
# lectures have credits > 0 associated with them, discussions seem to have QZ as credits
class_heading_tables = soup.find_all("table", attrs={"bgcolor": "#ffcccc"})

all_tables = soup.find_all("table")

# in all_tables list get index where class heading table starts
start_index = 0
while all_tables[start_index] != class_heading_tables[0]:
    start_index += 1

# dictionary with key as class heading table and values as list of tables between current class
# heading and next class heading
associated_tables = {}

current_class_table = 0
current_table = start_index + 1
current_class_associated_tables = []

while current_table < len(all_tables):
    # add class heading table to dictionary if no associated classes yet, empty list
    if len(current_class_associated_tables) == 0:
        associated_tables[class_heading_tables[current_class_table]] = []

    if current_class_table < len(class_heading_tables) - 1:
        if not all_tables[current_table] == class_heading_tables[current_class_table + 1]:
            current_class_associated_tables.append(all_tables[current_table])
        else:
            # add associated tables to current class heading table in dict
            associated_tables[class_heading_tables[current_class_table]] = current_class_associated_tables
            # clear out associated tables because moving on to new class
            current_class_associated_tables = []
            # increase current class table
            current_class_table += 1
    else: # on last class heading table
        # just add class tables to associated list and then add list to dict after while loop over
        current_class_associated_tables.append(all_tables[current_table])

    current_table += 1

# for last class heading
associated_tables[class_heading_tables[current_class_table]] = current_class_associated_tables


# print(text)

# group 1 - CSE
# group 2 - course number
# group 3 - course name, with a P at the end sometimes lol
course_search = "([A-Z]+)\s+(\d+)\s+((?:[A-Z]+\s*)+)"

# group 1 - credits either listed as 1, 1-1, or QZ
# group 2 - seats filled
# group 3 - seat capacity
section_search = "\d+\s+[A-Z0-9]+\s+([A-Z]+|\d+-\d+|\d+).*\s+(\d+)/\s+(\d+)"


class_codes = []
class_names = []
class_filled = []
class_capacity = []

for course in associated_tables:
    section_tables = associated_tables[course]
    
    heading_matches = re.search(course_search, course.text)
    print("{} {} {}".format(heading_matches.group(1), heading_matches.group(2), heading_matches.group(3)))
    
    for section in section_tables:
        section_matches = re.search(section_search, section.text)
        section_credits = section_matches.group(1)
        # checking that credits are numbers to indicate that it's a lecture, not discussion
        credits_num_search = re.search("\d+-\d+|\d+", section_credits)
        if credits_num_search:
            print("{} / {}".format(int(section_matches.group(2)), int(section_matches.group(3))))
            class_codes.append(heading_matches.group(1) + " " + heading_matches.group(2))
            class_names.append(heading_matches.group(3))
            class_filled.append(int(section_matches.group(2)))
            class_capacity.append(int(section_matches.group(3)))

classes_df = pd.DataFrame({"class_code": class_codes, "class_name": class_names, "class_filled": class_filled, "class_capacity": class_capacity})

classes_df.to_csv("uwash.csv")

Here is what the generated csv file looks like in a dataframe:

In [11]:
uwash = pd.read_csv("class_sizes_data/uwash.csv")
# unnamed column generated for some reason so delete it
uwash = uwash.loc[:, ~uwash.columns.str.contains('^Unnamed')]
uwash

,class_code,class_name,class_filled,class_capacity
0,CSE 142,COMPUTER PRGRMNG I,519,555
1,CSE 142,COMPUTER PRGRMNG I,285,355
2,CSE 143,COMPUTER PRGRMNG II,330,400
3,CSE 143,COMPUTER PRGRMNG II,135,235
4,CSE 143,COMPUTER PRGRMNG II,63,72
...,...,...,...,...
167,CSE 599,SPECIAL TOPICS,0,40
168,CSE 600,INDEPNDNT STDY,307,95
169,CSE 601,INTERNSHIP,78,70
170,CSE 700,MASTERS THESIS,0,30


#### University of Michigan

A Michigan CS professor created [Crapuler](http://crapuler.com) to help Michigan students create their class schedules. This website has a special feature if you go to http://crapuler.com/eecs which lists all the CS courses. This page is really helpful because it lists the term the course happened, the course code, and how many students were enrolled in the course. The only problem is that the course enrollment totals on this page sum all of the lecture totals. I instead want the counts per lecture per class. To do this, I had to visit this page, extract all the links to the classes, visit those class pages, then extract the lecture data.

Below is the code which does this:

In [ ]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import pandas as pd
import re

url = "http://crapuler.com/eecs?logname=Spring%2FSummer%2FFall+2021"

session = HTMLSession()

res = session.get(url)
res.html.render(sleep=1)

soup = BeautifulSoup(res.text, 'html.parser')

rows = soup.find("table").find_all("tr")

course_codes = []
course_links = []
course_types = []

for row in rows:
    data = row.find_all("td")
    
    i = 0
    is_fall_2021 = True
    while i < len(data) and is_fall_2021:
        if data[i].span.has_attr("title"):
            if data[i].span["title"] == "Term":
                # making sure fall 2021 course, if not go to next row
                if not "Fall 2021" in data[i].span.text:
                    is_fall_2021 = False
            elif data[i].span["title"] == "Enrollment History":
                # go up one element to get link
                link = row.find("a", href=True)
                course_codes.append(link.text)
                course_links.append("http://crapuler.com/" + link["href"])
            elif data[i].span["title"] == "Section Type":
                course_types.append(data[i].span.text)

        i += 1


# loop over course links to get individual section data

# lists for the dataframe 
course_codes_sections = []
course_types_sections = []
course_instructors_sections = []
course_times_sections = []
course_enrollments = []
course_capacities = []

for i, link in enumerate(course_links):
    res = session.get(link)
    res.html.render(sleep=2)

    soup = BeautifulSoup(res.text, 'html.parser')

    # assuming lecture table always first
    lecture_table = soup.find("table")

    # get table rows
    rows = lecture_table.find_all("tr")

    # check that this is lecture table
    if "Lectures" in rows[0].text:
        # loop rows, skip first 2 which don't have data we want
        for j in range(2, len(rows)):
            # get row data
            row_data = rows[j].find_all("td")

            course_codes_sections.append(course_codes[i])
            course_types_sections.append(course_types[i])
            
            # get instructor names
            instructor_links = row_data[1].find_all("a")
            instructors = []
            for instructor in instructor_links:
                instructors.append(instructor.text.strip())
            # sort so get consistent list of instructor names across sections if
            # need to combine some
            instructors.sort()
            course_instructors_sections.append(instructors)

            # remove whitespace for time
            time = re.sub("\s+", '', row_data[2].text)
            course_times_sections.append(time)

            # assuming enrolled always at index 4
            course_enrollments.append(int(row_data[4].text))
            # assuming capacity always at index 5
            course_capacities.append(int(row_data[5].text))
    else:
        print("at {}, lecture table not first".format(link))



classes_df = pd.DataFrame({"course_code": course_codes_sections, "course_type": course_types_sections, "course_enrollment": course_enrollments, "course_capacity": course_capacities, "course_time": course_times_sections, "course_instructors": course_instructors_sections})
classes_df.to_csv("umich.csv")

In [12]:
michigan = pd.read_csv("class_sizes_data/umich.csv")
michigan

,Unnamed: 0,course_code,course_type,course_enrollment,course_capacity,course_time,course_instructors
0,0,EECS 183,Lecture,220,245,TuTh8:30AM-10:00AM,"['Tabassum, Nuzhat', 'Torralva, Ben']"
1,1,EECS 183,Lecture,189,192,TuTh11:30AM-1:00PM,"['Kloosterman, John Sidney', 'Tabassum, Nuzhat']"
2,2,EECS 183,Lecture,250,250,TuTh1:00PM-2:30PM,"['Arthur, William', 'Tabassum, Nuzhat']"
3,3,EECS 183,Lecture,224,225,TuTh2:30PM-4:00PM,"['Kloosterman, John Sidney', 'Tabassum, Nuzhat']"
4,4,EECS 183,Lecture,257,260,TuTh4:00PM-5:30PM,"['Arthur, William', 'Tabassum, Nuzhat']"
...,...,...,...,...,...,...,...
244,244,ENGR 101,Lecture,182,180,TBATBA,"['Burdick, Laura']"
245,245,ENGR 101,Lecture,176,172,TBATBA,"['Alford, Laura']"
246,246,ENGR 101,Lecture,181,176,TBATBA,"['Alford, Laura']"
247,247,ENGR 101,Lecture,185,184,TBATBA,"['Alford, Laura']"


##### Cleaning Michigan Data

This looks good, however I noticed that some lectures would be at the same time with the same professors which mean that they should be combined into one lecture. I did this with the following code.

In [ ]:
import pandas as pd

classes_df = pd.read_csv("umich/umich.csv")

# if row course_code, course_time, and course_instructors are the same, 
# combine rows into one row

# ex class eecs 445

grouped = classes_df.groupby(["course_code", "course_time", "course_instructors"])

result = grouped.agg({"course_enrollment": "sum", "course_capacity": "sum"})

result.to_csv("umich_combined.csv")

The following output is the cleaned Michigan data:

In [13]:
michigan = pd.read_csv("class_sizes_data/umich_combined.csv")
michigan

,course_code,course_time,course_instructors,course_enrollment,course_capacity
0,EECS 183,TuTh11:30AM-1:00PM,"['Kloosterman, John Sidney', 'Tabassum, Nuzhat']",189,192
1,EECS 183,TuTh1:00PM-2:30PM,"['Arthur, William', 'Tabassum, Nuzhat']",250,250
2,EECS 183,TuTh2:30PM-4:00PM,"['Kloosterman, John Sidney', 'Tabassum, Nuzhat']",224,225
3,EECS 183,TuTh4:00PM-5:30PM,"['Arthur, William', 'Tabassum, Nuzhat']",257,260
4,EECS 183,TuTh8:30AM-10:00AM,"['Tabassum, Nuzhat', 'Torralva, Ben']",220,245
...,...,...,...,...,...
189,EECS 598,We4:30PM-5:30PM,"['Hofmann, Heath', 'Liu, Mingyan']",53,80
190,EECS 644,MoWe10:00AM-11:30AM,"['Polk, Thad']",14,12
191,ENGR 101,TBATBA,"['Alford, Laura']",542,532
192,ENGR 101,TBATBA,"['Burdick, Laura']",182,180


#### University of California Berkeley

I used Berkley's public class registry https://classes.berkeley.edu to get class size data. I made a query to get all CS courses and the registrar returned several pages of data. An interesting thing about Berkley's registrar is that it returns all of the data in JSON format which is great for me as a programmer. This made extracting data super easy.

One assumption I'm making is that each of these class listings is taught by 1 professor. I confirmed that this is a fair assumption to make since my friend at Berkeley said there may be 2 professors listed but only 1 is teaching at a time, or that 1 is the main professor.

The difficulty came when some CS classes are cross listed with other departments. The CS class listing would have 0 students enrolled and all of the students would be enrolled under the other department name. To remedy this problem, I had to find the cross listed classes links in the CS class json and visit those pages individually to extract the class enrollment data.

Below is the code which does all this:

In [ ]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession
import json
import time
import pandas as pd

url = "https://classes.berkeley.edu/search/class?page="
url2 = "&f[0]=im_field_subject_area%3A483&f[1]=im_field_term_name%3A2208"
pages = 5
session = HTMLSession()

course_names = []
course_codes = []
course_types = []
course_enrollments = []
course_capacities = []

for i in range(0, pages):
    res = session.get(url + str(i) + url2)
    res.html.render(sleep=1)

    soup = BeautifulSoup(res.text, 'html.parser')

    results = soup.find_all("li", {"class": "search-result"})


    results_data = []

    for result in results:
        # cross listed courses in tag data-json, if not cross listed, will not be key in json
        # data-json has combined enrollment counts, and enrollment for just cs version
        # course link in tag data-node, but not needed since already have enrollment counts in data-json
        results_data.append(json.loads(result.div['data-json']))

    # key - course code
    # value - list of cross listed course links so can visit pages later and enrollment counts
    cross_listing_links = {}

    for data in results_data:
        course_names.append(data["class"]["course"]["title"])
        course_codes.append(data["class"]["course"]["displayName"])
        course_types.append(data["component"]["code"])
        course_enrollments.append(data["enrollmentStatus"]["enrolledCount"])
        course_capacities.append(data["enrollmentStatus"]["maxEnroll"])

        if "crossListing" in data:
            links = []
            for attribute, value in data["crossListing"].items():
                links.append("https://classes.berkeley.edu/" + data["crossListing"][attribute]["path"])

            cross_listing_links[data["class"]["course"]["displayName"]] = links

    # handle cross listed courses by getting their enrollment info by visiting the links

    for course_code in cross_listing_links:
        for link in cross_listing_links[course_code]:
            res = session.get(link)
            res.html.render(sleep=1)
            soup = BeautifulSoup(res.text, 'html.parser')
            data = json.loads(soup.find("div", {"class": "handlebarData theme_is_whitehot"})["data-json"])
            
            # add cross listed course data
            course_names.append(data["class"]["course"]["title"])
            course_codes.append(data["class"]["course"]["displayName"])
            course_types.append(data["component"]["code"])
            course_enrollments.append(data["enrollmentStatus"]["enrolledCount"])
            course_capacities.append(data["enrollmentStatus"]["maxEnroll"])

            time.sleep(1)

    # sleep for each page
    time.sleep(1)


classes_df = pd.DataFrame({"course_code": course_codes, "course_type": course_types, "course_name": course_names, "course_enrollment": course_enrollments, "course_capacity": course_capacities})
classes_df.to_csv("ucb.csv")

Here is what the csv and corresponding dataframe look like:

In [15]:
berkeley = pd.read_csv("class_sizes_data/ucb.csv")
# unnamed column generated for some reason so delete it
berkeley = berkeley.loc[:, ~berkeley.columns.str.contains('^Unnamed')]
berkeley

,course_code,course_type,course_name,course_enrollment,course_capacity
0,COMPSCI C8,LEC,Foundations of Data Science,0,0
1,COMPSCI 10,LEC,The Beauty and Joy of Computing,370,350
2,COMPSCI 24,SEM,Freshman Seminars,0,12
3,COMPSCI 24,SEM,Freshman Seminars,13,15
4,COMPSCI 24,SEM,Freshman Seminars,20,20
...,...,...,...,...,...
79,COMPSCI 298,LEC,"Group Studies Seminars, or Group Research",22,28
80,COMPSCI 298,LEC,"Group Studies Seminars, or Group Research",11,20
81,COMPSCI 370,LEC,Adaptive Instruction Methods in Computer Science,10,80
82,COMPSCI 370,LEC,Adaptive Instruction Methods in Computer Science,28,80


##### Cleaning Berkeley Data

As you can see, some classes listed are not lectures and some have 0 enrollment. I need to filter for courses with non-zero enrollment and "LEC" type.

In [17]:
berkeley = berkeley[(berkeley["course_type"] == "LEC") & (berkeley["course_enrollment"] > 0)]
berkeley = berkeley.reset_index(drop=True)
berkeley

,course_code,course_type,course_name,course_enrollment,course_capacity
0,COMPSCI 10,LEC,The Beauty and Joy of Computing,370,350
1,COMPSCI 61A,LEC,The Structure and Interpretation of Computer P...,1666,1900
2,COMPSCI 61B,LEC,Data Structures,904,1000
3,COMPSCI 61C,LEC,Great Ideas of Computer Architecture (Machine ...,962,1175
4,COMPSCI 70,LEC,Discrete Mathematics and Probability Theory,785,850
5,COMPSCI 88,LEC,Computational Structures in Data Science,261,265
6,COMPSCI 161,LEC,Computer Security,609,625
7,COMPSCI 162,LEC,Operating Systems and System Programming,415,410
8,COMPSCI 164,LEC,Programming Languages and Compilers,124,128
9,DATA C8,LEC,Foundations of Data Science,1436,1440
